In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, concat_ws, pandas_udf
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

import pandas as pd
import numpy as np
import unidecode
import unicodedata
import re

In [2]:
spark = SparkSession.builder \
    .appName("Silver_BusIds") \
    .master("local[*]") \
    .getOrCreate()

In [3]:
spark

In [4]:
bus_ids = spark.read.csv("../raw/bus_company_ids.csv", header=True)

In [5]:
bus_ids.show()

+--------------------+----------+
|            Bus_Name|Company_ID|
+--------------------+----------+
|   Vũ Linh limousine|       768|
|             Ba Châu|     13078|
|            Văn Lang|     11826|
|            Trí Nhân|     12911|
|           Tuấn Hiệp|     11899|
|Thiện Thành Limou...|     21843|
|                 Hảo|     12797|
|  Trí Nhân Limousine|     12911|
|               Hà My|     12507|
|           Toàn Khải|     46289|
|           Giáp Diệp|     12772|
|             Tư Tiến|     13184|
|           Mạnh Hùng|     11188|
|           Ngọc Diễm|     40437|
|            Tân Niên|     12462|
|Duy Thảo (Kiên Gi...|     44740|
|             Dũng Lệ|     12944|
|            Hải Luân|     36137|
|           Liên Hưng|       808|
|Hùng Cường - An G...|       529|
+--------------------+----------+
only showing top 20 rows



In [6]:
import re

def remove_accents(text):
    if text is None:
        return ""
    # Thay thế riêng biệt chữ "đ" và "Đ" trước khi chuẩn hóa
    text = text.replace("đ", "d").replace("Đ", "D")
    # Chuẩn hóa và loại bỏ dấu tiếng Việt
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    # Thay thế các ký tự đặc biệt (ngoại trừ chữ, số) bằng dấu gạch ngang "-"
    text = re.sub(r"[^a-zA-Z0-9]+", "-", text)
    # Xóa dấu gạch ngang ở đầu và cuối nếu có
    return text.strip("-").lower()

# Tạo UDF mới
remove_accents_udf = udf(remove_accents, StringType())

In [7]:
bus_ids = bus_ids.withColumn("Bus_Id", concat_ws("_", remove_accents_udf(col("Bus_Name")), col("Company_Id"))).select("Bus_Id", "Bus_Name", "Company_Id")

In [8]:
# Hiển thị kết quả
bus_ids.show()

+--------------------+--------------------+----------+
|              Bus_Id|            Bus_Name|Company_Id|
+--------------------+--------------------+----------+
|vu-linh-limousine...|   Vũ Linh limousine|       768|
|       ba-chau_13078|             Ba Châu|     13078|
|      van-lang_11826|            Văn Lang|     11826|
|      tri-nhan_12911|            Trí Nhân|     12911|
|     tuan-hiep_11899|           Tuấn Hiệp|     11899|
|thien-thanh-limou...|Thiện Thành Limou...|     21843|
|           hao_12797|                 Hảo|     12797|
|tri-nhan-limousin...|  Trí Nhân Limousine|     12911|
|         ha-my_12507|               Hà My|     12507|
|     toan-khai_46289|           Toàn Khải|     46289|
|     giap-diep_12772|           Giáp Diệp|     12772|
|       tu-tien_13184|             Tư Tiến|     13184|
|     manh-hung_11188|           Mạnh Hùng|     11188|
|     ngoc-diem_40437|           Ngọc Diễm|     40437|
|      tan-nien_12462|            Tân Niên|     12462|
|duy-thao-